# SQLAlchemy 사용해서 엔진 만들기

In [1]:
# 클래스랑 실제 table랑 자동으로 매핑해주는 애.
# 복잡하게 테이블 만들어서 작업하지말고, 중간에서 ORM이 알아서 해줄테니, 편하게 class로 다 해라.

# ER모델과 실제 테이블과 정확하게 매핑되지 않는 경우 있었는데,
# ORM에서는 그런경우가 없음. class만들면 알아서 table만들어주니까!!!

# class끼리 상속, abstract, 모든 관계 갖다쓸 수 있음.
# DBMS는 아님..! RDBMS 개체의 관계를 엮을 수 있게 해주는 프로그램 테크닉일뿐.

# ORM 밑에 RDBMS가 반드시 존재해야함.
# database랑 object랑 자동으로 mapping해주기때문에 개발자에겐 쉬움.
# ER 모델과 비지니스모델(객체모델) 과 달라서 고민하는 경우가 없음.
# But, 중간에 ORM을 거치므로, overhead가 발생해서 성능이 좀 떨어짐. but, transaction이 심하지 않은곳에서는 사용할 수 있음.
# 1000개의 데이터를 10000번씩 불러올땐 문제가 될 순 있음.
# DB를 안쓰는이융: 하나쿼리를 실행해서 이름하나 불러오는데 0.1초 걸린다 해도, 시간 엄청많이 걸리게됨.
# 따라서, 저분은 fileDB로 덤프를 통째로 가져온다고 함.

In [2]:
!conda list

# !pip list
# sqlAlchemy 깔려있는지 확인.

# packages in environment at C:\Users\user\Anaconda3:
#
# Name                    Version                   Build  Channel
_ipyw_jlab_nb_ext_conf    0.1.0            py36he6757f0_0  
alabaster                 0.7.10           py36hcd07829_0  
anaconda                  5.2.0                    py36_3  
anaconda-client           1.6.14                   py36_0  
anaconda-navigator        1.8.7                    py36_0  
anaconda-project          0.8.2            py36hfad2e28_0  
asn1crypto                0.24.0                   py36_0  
astroid                   1.6.3                    py36_0  
astropy                   3.0.2            py36h452e1ab_1  
attrs                     18.1.0                   py36_0  
babel                     2.5.3                    py36_0  
backcall                  0.1.0                    py36_0  
backports                 1.0              py36h81696a8_1  
backports.shutil_get_terminal_size 1.0.0            py36h79ab834_2  
beautifulsoup4            4.

In [3]:
import sqlalchemy

In [4]:
sqlalchemy.__version__

'1.2.7'

In [9]:
# 엔진을 만드는 애 불러오기.
# fileDB를쓰니까 쓸게별로없는데, 다른DB쓰려면 localhost, 계정이름, 포트번호, 어떤DB에 접속할지(DB번호)도 주소에 적어줘야함.
# sqlLite는 파일DB형식이라 host가 없음.
# 경로가 간단함.
# sqlite:///:memory:     메모리에서 작업할때.
# sqlite:///
# sqlite:///file.db      파일DB연동.

from sqlalchemy import create_engine

engine = create_engine("sqlite://", echo=True)
# 실제 DB에 들어간건지 어쩐지 에코를 주면 덜 답답하니까.

In [10]:
print(engine)

Engine(sqlite://)


In [13]:
from sqlalchemy import MetaData

metadata = MetaData()
print(metadata)

MetaData(bind=None)


In [15]:
from sqlalchemy import Table, Column, Integer, String, ForeignKey

#테이블 객체(인스턴스) 만들기
# 테이블명/ metadata(오브젝트를 담고있는애.)를 상속받아서 users만들거./ column객체(컬럼명,data타입,primary키인지아닌지,)써서 컬럼 만들거임.
users = Table('users', metadata,
             Column('id', Integer, primary_key="True"),
             Column('name', String),
             Column('fullname', String))

In [16]:
print(users)

users


In [17]:
# user_id는 컬럼이름일 뿐. ForeignKey("참조대상") 참조대상에 컬럼명쓰지않도록 주의!
# sqlAlchemy의 core단에서 객체로 만들었음.

address = Table('addresses', metadata,
               Column('id', Integer, primary_key = "True"),
               Column('user_id', Integer, ForeignKey('users.id')),
               Column('email', String, nullable=False))

In [22]:
# 메타데이터랑 매핑.
#메타데이터엔 우리가만든 객체들이잇음.
# 엔진에게 metadata랑 매핑해서 sql문만들라고 명령.
#엔진에다가 매핑시킴.
#엔진에다가 또 알려줘야함 계속. 다른걸할때마다.
# 자동으로 sql문만들어짐. 
# ORM배우면 더 놀랄거임. 한단계 더 추상화되어있기때문에....

metadata.create_all(engine)

2018-07-10 13:46:24,782 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2018-07-10 13:46:24,783 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 13:46:24,785 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("addresses")
2018-07-10 13:46:24,787 INFO sqlalchemy.engine.base.Engine ()


In [24]:
for row in metadata.tables:
    print(row)
    
print(metadata)

users
addresses
MetaData(bind=None)


# insert 실행하기 (1)
### users 객체의 insert().values() 으로 실행하기

In [25]:
print(users.insert())

INSERT INTO users (id, name, fullname) VALUES (:id, :name, :fullname)


In [30]:
# fullname은 table의 컬럼이 아니고, class의 attribute임.
# attribute가 아니라면 ""로 감쌌어야함.

insert = users.insert().values(name="kim", fullname="Anonymous, Kim")

In [31]:
print(users.insert().values(name="kim", fullname="Anonymous, Kim"))

# 두개의 테이블 attribute를 넣어놨으니 자연스럽게 column으로 매핑되고, values들이 들어감.

INSERT INTO users (name, fullname) VALUES (:name, :fullname)


In [32]:
#컴파일해야만 sql문으로 바뀜.

insert.compile()

In [33]:
# 실제 어떻게 바뀌어서 들어왔는지 알수있음.

insert.compile().params

{'name': 'kim', 'fullname': 'Anonymous, Kim'}

In [34]:
# 바인딩 시켜보면.
# 엔진을 줄거에요.
# 엔진에게 메모리에서 sqlite작업할거다라고 했고.
#실제 커넥트에서 하나 받아오면됨.

insert.bind = engine

In [35]:
conn = engine.connect()

In [40]:
# 엔진에서 받아와서 엔진에 잇는 insert문을 실행시킴.
# sql문으로 해석되가지고. 나가는거임.

result = conn.execute(insert)
#실행결과 보여줌.

2018-07-10 13:54:57,736 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2018-07-10 13:54:57,738 INFO sqlalchemy.engine.base.Engine ('kim', 'Anonymous, Kim')
2018-07-10 13:54:57,740 INFO sqlalchemy.engine.base.Engine COMMIT


In [41]:
#제일마지막에 들어간 key값을 가져오기.
result.inserted_primary_key

[3]

# insert문 실행하기(2)
### connect 객체의 execute()으로 실행하기

In [46]:
insert = users.insert()

In [47]:
result = conn.execute(insert, name="lee", fullname="Unknow, Lee")

# 엔진이 어떻게 해석했는지 echo로 다 보여줌.

2018-07-10 14:13:40,420 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2018-07-10 14:13:40,421 INFO sqlalchemy.engine.base.Engine ('lee', 'Unknow, Lee')
2018-07-10 14:13:40,422 INFO sqlalchemy.engine.base.Engine COMMIT


In [48]:
result.inserted_primary_key

[5]

In [50]:
result = conn.execute(insert, name="xxx", fullname="xxxx, XXX")

2018-07-10 14:14:16,609 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2018-07-10 14:14:16,610 INFO sqlalchemy.engine.base.Engine ('xxx', 'xxxx, XXX')
2018-07-10 14:14:16,611 INFO sqlalchemy.engine.base.Engine COMMIT


In [51]:
result.inserted_primary_key

[6]

In [85]:
# 총 2개를 넣을껀데, dictionary형태로 넣을것임.
# insert 문 2개 실행한셈.
# Lower단에서 executeMany로한거랑 같음.

conn.execute(address.insert(), [
    {'user_id' : 1, "email" : "anonymous.kim@test.com"},
    {'user_id' : 2, "email" : "unknown.lee@test.com"}
]);

2018-07-10 14:26:07,152 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (user_id, email) VALUES (?, ?)
2018-07-10 14:26:07,153 INFO sqlalchemy.engine.base.Engine ((1, 'anonymous.kim@test.com'), (2, 'unknown.lee@test.com'))
2018-07-10 14:26:07,154 INFO sqlalchemy.engine.base.Engine COMMIT


# select문 실행하기
#### select(), where(), fetchone(), fetchall()

In [151]:
# resultProxy를 쓰기위해 작업을 할거임....!

from sqlalchemy.sql import select

query = select([users])
print(query)

SELECT users.id, users.name, users.fullname 
FROM users


In [157]:
result = conn.execute(query)

# 엔진이 실제 해석해서 나온 명령문이 아래와 같음.

2018-07-10 14:52:36,213 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2018-07-10 14:52:36,214 INFO sqlalchemy.engine.base.Engine ()


In [153]:
for row in result:
    print(row)

(1, 'kim', 'Anonymous, Kim')
(2, 'kim', 'Anonymous, Kim')
(3, 'kim', 'Anonymous, Kim')
(4, 'lee', 'Unknow, Lee')
(5, 'lee', 'Unknow, Lee')
(6, 'xxx', 'xxxx, XXX')


In [161]:
print(row)

(6, 'xxx', 'xxxx, XXX')


In [156]:
row = result.fetchone()

#????????????????? 왜안나옴?
# => ㅇㅋㅇㅋ print() 도 cursor역할을 한당.

In [160]:
print(row)

(6, 'xxx', 'xxxx, XXX')


In [158]:
rows = result.fetchall()

In [159]:
for row in rows:
    print(row)

(1, 'kim', 'Anonymous, Kim')
(2, 'kim', 'Anonymous, Kim')
(3, 'kim', 'Anonymous, Kim')
(4, 'lee', 'Unknow, Lee')
(5, 'lee', 'Unknow, Lee')
(6, 'xxx', 'xxxx, XXX')


In [93]:
for row in rows:
    print(row[0], row[1], row[2])
    
# 처번째 컬럼.

In [162]:
# * 로 모든 컬럼가져오는 게 아니라, 특정컬럼 가져오고싶을때.
# 바로 못가져옴. 컬럼을 의미하는 c가 필요함.
# Column인스턴스가 감싸고 있기때문에

result = conn.execute(select([users.c.name, users.c.fullname]))

2018-07-10 14:55:28,046 INFO sqlalchemy.engine.base.Engine SELECT users.name, users.fullname 
FROM users
2018-07-10 14:55:28,047 INFO sqlalchemy.engine.base.Engine ()


In [163]:
for row in result:
    print(row)

('kim', 'Anonymous, Kim')
('kim', 'Anonymous, Kim')
('kim', 'Anonymous, Kim')
('lee', 'Unknow, Lee')
('lee', 'Unknow, Lee')
('xxx', 'xxxx, XXX')


In [164]:
row = result.fetchone()
print(row)

None


In [106]:
# PPT.142) Conjunctions
# where절을 구성하는  add, or, not을 가져올것임.

from sqlalchemy import and_, or_, not_

In [104]:
# Python이므로 == 표현이 가능함.

print(users.c.id == address.c.user_id)
# Python구문이 어떻게 sql문으로 바뀌는지 확인함.

users.id = addresses.user_id


In [112]:
or_str = or_(users.c.id == address.c.user_id, users.c.id == 1)

In [113]:
print(or_str)

users.id = addresses.user_id OR users.id = :id_1


In [114]:
# or_안쓰고 논리연산자 OR만가지고 Python구문만으로도 쿼리문생성가능.

print((users.c.id == address.c.user_id) | (users.c.id==1))

users.id = addresses.user_id OR users.id = :id_1


In [115]:
# 함수와 함수를 pipeline으로 연결만해주면된다.
# select, where순차적으로 진행해주기만하면됨.
# engine에서 만든 connection이라는 객체를 통해 execute를 한다.
result = conn.execute(select([users]).where(users.c.id==1))

# qmark스타일로 했네.
# metadata에서 engine바인딩해서, insert 
# compile해서 param값 받아오고. result에 실제 값잇을거고.

2018-07-10 14:37:56,167 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users 
WHERE users.id = ?
2018-07-10 14:37:56,169 INFO sqlalchemy.engine.base.Engine (1,)


In [117]:
for row in result:
    print(row)

(1, 'kim', 'Anonymous, Kim')


In [118]:
#join을 해야하는데, 귀찮으니까 where절 사용.
#cross join이지만, 실제로 가져온 형태는 innerjoin. 교집합만 가져온 형태.
# pk랑 fk랑 묶어봄.

result = conn.execute(select([users, address]).where(users.c.id==address.c.user_id))

2018-07-10 14:42:19,761 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname, addresses.id, addresses.user_id, addresses.email 
FROM users, addresses 
WHERE users.id = addresses.user_id
2018-07-10 14:42:19,762 INFO sqlalchemy.engine.base.Engine ()


In [119]:
for row in result:
    print(row)
    
# users부터. address까지.
# join문법몰라도, where절만가지고도 충분히 사용 가능하긴함. 좀이따 where대신 join도 배울거임.

(1, 'kim', 'Anonymous, Kim', 1, 1, 'anonymous.kim@test.com')
(2, 'kim', 'Anonymous, Kim', 2, 2, 'unknown.lee@test.com')
(1, 'kim', 'Anonymous, Kim', 3, 1, 'anonymous.kim@test.com')
(2, 'kim', 'Anonymous, Kim', 4, 2, 'unknown.lee@test.com')


In [123]:
# users테이블의 name이 1번이 kim일거니까. k로 시작하는 모든것들을 불러오기.

result = conn.execute(select([users]).where(users.c.name.like("k%")))

2018-07-10 14:46:05,259 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users 
WHERE users.name LIKE ?
2018-07-10 14:46:05,260 INFO sqlalchemy.engine.base.Engine ('k%',)


In [124]:
for row in result:
    print(row)

(1, 'kim', 'Anonymous, Kim')
(2, 'kim', 'Anonymous, Kim')
(3, 'kim', 'Anonymous, Kim')


In [127]:
# where절로 묶어보기.
# AND를 통해 where절 2개가 묶임.

result = conn.execute(select([users, address]).where(users.c.id == address.c.user_id).where(users.c.name.like("k%")))

2018-07-10 14:47:59,797 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname, addresses.id, addresses.user_id, addresses.email 
FROM users, addresses 
WHERE users.id = addresses.user_id AND users.name LIKE ?
2018-07-10 14:47:59,798 INFO sqlalchemy.engine.base.Engine ('k%',)


In [128]:
for row in result:
    print(row)


(1, 'kim', 'Anonymous, Kim', 1, 1, 'anonymous.kim@test.com')
(2, 'kim', 'Anonymous, Kim', 2, 2, 'unknown.lee@test.com')
(1, 'kim', 'Anonymous, Kim', 3, 1, 'anonymous.kim@test.com')
(2, 'kim', 'Anonymous, Kim', 4, 2, 'unknown.lee@test.com')


# 조인 (sqLite는 left join만 지원함)

In [166]:
from sqlalchemy import join

In [168]:
print(users.join(address))

# 누구랑 관련이있기에 join을 하느냐 on절 따로 안써줘도 됨. fk를 명시적으로 써줬기때문에. 

users JOIN addresses ON users.id = addresses.user_id


In [170]:
# onclause로 명시해도됨. 

print(users.join(address, users.c.id == address.c.user_id))

users JOIN addresses ON users.id = addresses.user_id


In [171]:
# select_from 을 통해 조인된 애들로부터 가져오겠다.
# users 와 address랑 조인한 형태의 테이블에서 가져오기.
# fk, pk를 지정해놓은 바람에 훨씬 간단해짐.
# 조인이아까보다 쉬워진거랭.

result = conn.execute(select([users]).select_from(users.join(address)))

2018-07-10 15:09:58,988 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users JOIN addresses ON users.id = addresses.user_id
2018-07-10 15:09:58,990 INFO sqlalchemy.engine.base.Engine ()


In [172]:
for row in result:
    print(row)

(1, 'kim', 'Anonymous, Kim')
(2, 'kim', 'Anonymous, Kim')
(1, 'kim', 'Anonymous, Kim')
(2, 'kim', 'Anonymous, Kim')
